In [36]:
# import all the modules

import os
import csv
import requests
from zipfile import ZipFile
from io import BytesIO, TextIOWrapper
from urllib.request import urlopen
import pandas as pd
from bs4 import BeautifulSoup
import matplotlib.pylab as plt


In [60]:
# grab zip file MHSA dataset
response = requests.get('https://arlweb.msha.gov/OpenGovernmentData/DataSets/Accidents.zip')

with ZipFile(BytesIO(response.content)) as zf:
    filename = zf.namelist()
    with zf.open(filename[0]) as file:
        df = pd.read_csv(file, on_bad_lines='skip', encoding='ISO-8859-1', delimiter='|')
# open zip file
#with ZipFile(BytesIO(response.read())) as zf:
    # open text file
    #with zf.open([i for i in zf.namelist() if ".txt" in i][0], 'r') as infile:
        # make it a dataframe
        #df = pd.read_csv(infile, delimiter='|')

df.head()

C:\Users\Jessie\AppData\Local\Temp\ipykernel_46096\4254819042.py:7: DtypeWarning: Columns (35) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, on_bad_lines='skip', encoding='ISO-8859-1', delimiter='|')


,MINE_ID,CONTROLLER_ID,CONTROLLER_NAME,OPERATOR_ID,OPERATOR_NAME,CONTRACTOR_ID,DOCUMENT_NO,SUBUNIT_CD,SUBUNIT,ACCIDENT_DT,...,DAYS_RESTRICT,DAYS_LOST,TRANS_TERM,RETURN_TO_WORK_DT,IMMED_NOTIFY_CD,IMMED_NOTIFY,INVEST_BEGIN_DT,NARRATIVE,CLOSED_DOC_NO,COAL_METAL_IND
0,100003,0041044,Lhoist Group,L13586,"Lhoist North America of Alabama, LLC",MPH,220132310045,3,"STRIP, QUARY, OPEN PIT",08/12/2013,...,2.0,0,N,08/19/2013,?,NO VALUE FOUND,NaN,Employee says he slipped off back of fuel truc...,NaN,M
1,100003,0041044,Lhoist Group,L13586,"Lhoist North America of Alabama, LLC",NaN,220241210019,30,MILL OPERATION/PREPARATION PLANT,04/22/2024,...,0.0,0,N,04/22/2024,?,NO VALUE FOUND,NaN,While blowing lime dust from the lime cooler a...,NaN,M
2,100009,M00024,Legacy Vulcan Corp (Form:Vulcan Materials Co),L16168,"Vulcan Construction Materials, LLC",NaN,220053250019,3,"STRIP, QUARY, OPEN PIT",11/13/2005,...,13.0,0,N,12/01/2005,?,NO VALUE FOUND,NaN,2 employees were attempting to load troughing ...,320053360009,M
3,100011,M11763,Imerys S A,L17074,Imerys Pigments LLC,NaN,220062000052,30,MILL OPERATION/PREPARATION PLANT,07/14/2006,...,0.0,0,N,07/15/2006,?,NO VALUE FOUND,NaN,Employee was pushing against pry bar. Bar slip...,NaN,M
4,100011,M11763,Imerys S A,0090005,Imerys Carbonates LLC,NaN,220090120006,30,MILL OPERATION/PREPARATION PLANT,01/08/2009,...,0.0,0,N,01/09/2009,?,NO VALUE FOUND,NaN,Employee tripped on c-channel at #6 silo. Fel...,NaN,M
